# Research FFD Experiment 1

In this notebook we show how to run experiment 1 in local C1-W4 FFD intrastructure. Please run the FFD-Data-Formatting notebook first to have the necessery data for the experiments. The necessery packages are:
- NumPy
- Pandas
- Matplotlib
- MinIO

In [1]:
import io
import pickle
import requests
import json

import numpy as np
import pandas as pd

from minio import Minio

## Setup

Please open 3 terminals and move them to FFD/code/research/deployment/compose. To setup the infrastructure, run the following commands in the separate terminals in this order:

```
# First terminal
docker compose -f ffd-storage-docker-compose.yaml up

# Second terminal
docker compose -f ffd-experiment-monitoring-docker-compose.yaml up

# Third terminal
docker compose -f ffd-c1-w4-nodes-docker-compose.yaml up
```

Remember, that you can stop the containers with CTRL+C or use a separate terminal to stop or remove the containers with these commands:

```
docker compose -f ffd-storage-docker-compose.yaml stop
docker compose -f ffd-storage-docker-compose.yaml down

docker compose -f ffd-experiment-monitoring-docker-compose.yaml stop
docker compose -f ffd-experiment-monitoring-docker-compose.yaml down

docker compose -f ffd-c1-w4-nodes-docker-compose.yaml stop
docker compose -f ffd-c1-w4-nodes-docker-compose.yaml down
```

## Logs, Status, UIs and Dashboards

If there are no errors in the terminal logs, open the following addresses:

- Grafana: http://127.0.0.1:3000/
    - User = admin
    - Password = admin
- MLflow: http://127.0.0.1:5000/
- Central: http://127.0.0.1:7500/logs
- Worker-1: http://127.0.0.1:7501/logs
- Worker-2: http://127.0.0.1:7502/logs
- Worker-3: http://127.0.0.1:7503/logs
- Worker-4: http://127.0.0.1:7504/logs
- MinIO: http://127.0.0.1:9001/
    - User = 23034opsdjhksd
    - Password = sdkl3slömdm
- Prometheus: http://127.0.0.1:9090/

When you have opened all of these, check the logs of central and workers. If there are no constant errors and workers have registered themselves into central, then the infrastructure is ready to run a experiment. 

It is recommended to check container resource consumption with:

```
Docker stats
```

and use Grafana for data analysis by importing the code/research/deployment/grafana/ffd_dashboard.json to setup the dashboard.

## Starting Training

Please run these following blocks step by step to initilized training:

### Data

In [3]:
formated_data_df = pd.read_csv('../data/formated_fraud_fetection_data.csv')

In [8]:
formated_data_df

,step,amount,nameOrig,nameDest,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER,isFraud,isFlaggedFraud
0,1,9840,1,7804113,0,0,0,1,0,0,0
1,1,1864,2,8163007,0,0,0,1,0,0,0
2,1,181,3,6686021,0,0,0,0,1,1,0
3,1,181,4,7859954,0,1,0,0,0,1,0
4,1,11668,5,9012814,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,339682,6353303,8714971,0,1,0,0,0,1,0
6362616,743,6311409,6353304,6551940,0,0,0,0,1,1,0
6362617,743,6311409,6353305,7010883,0,1,0,0,0,1,0
6362618,743,850003,6353306,6942702,0,0,0,0,1,1,0


In [51]:
used_data = formated_data_df.iloc[:500000]

In [52]:
used_data

,step,amount,nameOrig,nameDest,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER,isFraud,isFlaggedFraud
0,1,9840,1,7804113,0,0,0,1,0,0,0
1,1,1864,2,8163007,0,0,0,1,0,0,0
2,1,181,3,6686021,0,0,0,0,1,1,0
3,1,181,4,7859954,0,1,0,0,0,1,0
4,1,11668,5,9012814,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
499995,20,77616,499949,7917483,0,1,0,0,0,0,0
499996,20,63262,499950,7800862,0,1,0,0,0,0,0
499997,20,15019,499951,8750535,0,0,0,1,0,0,0
499998,20,355629,499952,6672790,0,0,0,0,1,0,0


In [53]:
target_value_amounts = used_data['isFraud'].value_counts()
print(target_value_amounts)
fraud_to_no_fraud_ratio = target_value_amounts / target_value_amounts.sum()
print(fraud_to_no_fraud_ratio)

0    499767
1       233
Name: isFraud, dtype: int64
0    0.999534
1    0.000466
Name: isFraud, dtype: float64


In [54]:
data = used_data.values.tolist()
columns = formated_data_df.columns.tolist()

## Experiment Metadata

In [65]:
experiment = {
    'name': 'ffd-experiment-1',
    'tags': {}
}

## Model Parameters

In [66]:
model_parameters = {
    'seed': 42,
    'used-columns': [
        'amount',
        'type_CASH_IN',
        'type_CASH_OUT',
        'type_DEBIT',
        'type_PAYMENT',
        'type_TRANSFER',
        'isFraud'
    ],
    'input-size': 6,
    'target-column': 'isFraud',
    'scaled-columns': [
        'amount'
    ],
    'learning-rate': 0.20,
    'sample-rate': 0.20,
    'optimizer':'SGD',
    'epochs': 10
}

### Central Parameters

In [67]:
central_parameters = {
    'sample-pool': 250000,
    'data-augmentation': {
        'active': True,
        'sample-pool': 250000,
        '1-0-ratio': 0.2
    },
    'eval-ratio': 0.2,
    'train-ratio': 0.9,
    'min-update-amount':4,
    'max-cycles':5,
    'min-metric-success': 10,
    'metric-thresholds': {
        'true-positives': 2000,
        'false-positives': 1000,
        'true-negatives': 40000, 
        'false-negatives': 10000,
        'recall': 0.20,
        'selectivity': 0.90,
        'precision': 0.80,
        'miss-rate': 0.50,
        'fall-out': 0.05,
        'balanced-accuracy': 0.70,
        'accuracy': 0.80
    },
    'metric-conditions': {
        'true-positives': '>=',
        'false-positives': '<=',
        'true-negatives': '>=', 
        'false-negatives': '<=',
        'recall': '>=',
        'selectivity': '>=',
        'precision': '>=',
        'miss-rate': '<=',
        'fall-out': '<=',
        'balanced-accuracy': '>=',
        'accuracy': '>='
    }
}

## Worker Parameters

In [68]:
worker_parameters = {
    'sample-pool': 250000,
    'data-augmentation': {
        'active': True,
        'sample-pool': 250000,
        '1-0-ratio': 0.2
    },
    'eval-ratio': 0.2,
    'train-ratio': 0.9
}

### Context Payload

In [69]:
parameters = {
    'model': model_parameters,
    'central': central_parameters,
    'worker': worker_parameters
}

context = {
    'experiment': experiment,
    'parameters': parameters,
    'data': data,
    'columns': columns
}

payload = json.dumps(context)
print('Payload size in bytes: ' + str(len(payload)))

Payload size in bytes: 25445465


### Sending Context

In [70]:
response = requests.post(
    url = 'http://127.0.0.1:7500/start',
    json = payload
)

print(response.status_code)

200


## Checking training with Logs and Docker stats

During the run its recommeded to check central progress by updating the logs website and checking the metrics of docker stats, which you can read about [here]( https://docs.docker.com/reference/cli/docker/container/stats/). You know that the training is complete, when Central logs show evaluation results and Workers now longer receive any context from Central, causing their tasks to idle with False prints. 

## Nodes, MLflow and MinIO

Because **central and worker are stateless** due to using MinIO, when you see that the logs start to idle, you can stop node deployment with either CTRL+C or using docker compose -f (file) stop unless you want to try model inference. 

As a reminder it is not recommeded to use docker compose -f (file) down on storage and monitoring deployments, because **the gathered data will be lost**. **Data will not be wiped during computer restarts**, but to prevent the loss of MinIO and MLflow data simply download the buckets and runs to your computer by in MinIO case clicking a folder and checking the provided options on the right side and in MLflow case clicking all runs and clicking the 3 dots on the right side on top of new run button. 

## Checking results with MLflow and Grafana

In order to analyse model, time and resource metrics, the fastest way to get a general idea is to use MLflow comparison analysis and Grafana dashboard on a given time range. 

### MLflow

To analyse with MLflow simply click the experiments and runs you intende to analyse and then click compare, which provides different analysis options. For simplicity we recommed using the pararell coordinates plot to compare evaluation results in metrics list using the parameters list, which you can open and select by clicking the metrics and parameters you want. Its also recommeded to simply scroll down to get a list of the different metrics.  

### Grafana

To analyse with Grafana, press the three bars under grafana logo and go to dashboards. There press new, select import and select the ffd_dashboard.json provided in deployment/grafana. Now, you might need to correct the time range by selecting the top right date and setting the last two hours. When you see the plots having lines, simply click inside one of them to reduce the time range of all plots to the one you want.

## Comparing Results

In order to compare gathered results between docker stats, mlflow, grafana and minio, go to the images, data/mlflow and data/minio to compare the results of this experiment.